In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.linear_model import LogisticRegression
from lime.lime_text import LimeTextExplainer
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
config = GPT2Config.from_pretrained('gpt2-medium', num_labels=5)
model = GPT2ForSequenceClassification(config)
model.eval()


/home/xinyu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=1024, out_features=5, bias=False)
)

In [2]:
tokenizer.pad_token = tokenizer.eos_token
def predict_proba(texts):
    outputs = []
    predicted_classes = []
    for text in texts:
        inputs = tokenizer.encode_plus(text, return_tensors='pt', truncation=True, padding='max_length', max_length=tokenizer.model_max_length)
        with torch.no_grad():
            logits = model(**inputs).logits
        probabilities = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
        outputs.append(probabilities.squeeze(0))
        predicted_class = np.argmax(probabilities, axis=1)
        predicted_classes.append(predicted_class[0])
    return np.array(outputs),predicted_classes

In [3]:
from datasets import load_dataset
dataset = load_dataset("math_qa", split = "validation")

In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
explainer = LimeTextExplainer(class_names=["a", "b", "c", "d", "e"])

In [7]:
text_to_explain = dataset[0]["Problem"] + " " + dataset[0]["options"]
print(text_to_explain)

a multiple choice test consists of 4 questions , and each question has 5 answer choices . in how many r ways can the test be completed if every question is unanswered ? a ) 24 , b ) 120 , c ) 625 , d ) 720 , e ) 1024


In [13]:
for i in range(2,5):
    print(i)

2
3
4


In [ ]:
# Prompt 1: Problem + Options
num_example = 10
for i in range(5,num_example):
    text_to_explain = dataset[i]["Problem"] + " " + dataset[i]["options"]
    probabilities, predicted_classes = predict_proba([text_to_explain])
    exp = explainer.explain_instance(text_to_explain, lambda x: predict_proba(x)[0], num_features=6, num_samples=200, top_labels=5)
    exp.show_in_notebook(text=True,labels=[predicted_classes[0]])
    exp.show_in_notebook(text=False,labels=[0,1,2,3,4])


In [ ]:
exp.show_in_notebook(text=text_to_explain,labels=[0,1,2,3,4])

In [ ]:
# Prompt 2: Problem + Rationale + Options
num_example = 5
for i in range(num_example):
    text_to_explain = dataset[i]["Problem"] + " " + dataset[i]["Rationale"] + " " + dataset[i]["options"]
    exp = explainer.explain_instance(text_to_explain, lambda x: predict_proba(x)[0], num_features=6, num_samples=200, top_labels=5)
    exp.show_in_notebook(text=text_to_explain,labels=[predicted_classes[0]])
    exp.show_in_notebook(text=False,labels=[0,1,2,3,4])

In [ ]:
# Prompt 4: Problem + Annotated_formula + Options
num_example = 1
for i in range(num_example):
    text_to_explain = dataset[i]["Problem"] + " " + dataset[i]["annotated_formula"] + " " + dataset[i]["options"]
    exp = explainer.explain_instance(text_to_explain, lambda x: predict_proba(x)[0], num_features=6, num_samples=100, top_labels=5)
    exp.show_in_notebook(text=text_to_explain,labels=[predicted_classes[0]])
    exp.show_in_notebook(text=False,labels=[0,1,2,3,4])

In [ ]:
exp.show_in_notebook(text=text_to_explain,labels=(3,))

In [8]:
text_to_explain = 'a multiple choice test consists of 4 questions , and each question has 5 answer choices . in how many r ways can the test be completed if every question is unanswered ? "5 choices for each of the 4 questions , thus total r of 5 * 5 * 5 * 5 = 5 ^ 4 = 625 ways to answer all of them ." a ) 24 , b ) 120 , c ) 625 , d ) 720 , e ) 1024'
text_to_explain

'a multiple choice test consists of 4 questions , and each question has 5 answer choices . in how many r ways can the test be completed if every question is unanswered ? "5 choices for each of the 4 questions , thus total r of 5 * 5 * 5 * 5 = 5 ^ 4 = 625 ways to answer all of them ." a ) 24 , b ) 120 , c ) 625 , d ) 720 , e ) 1024'

In [ ]:
exp.show_in_notebook(text=text_to_explain,labels=[0])
exp.show_in_notebook(text=False,labels=[0,1,2,3,4])


In [27]:
dataset=['a 3 - digit positive integer is chosen at random . what is the probability that the product of its digits is even ? p ( three odd digits ) = 5 / 9 * 5 / 10 * 5 / 10 = 5 / 36 ( first digit can not be 0 , so we have a total of 9 digits , 4 even and 5 odd ) . therefore , p ( at least one even digit ) = 1 - 5 / 36 = 31 / 36 . a ) 1 / 2 , b ) 31 / 36 , c ) 49 / 54 , d ) 7 / 8 , e ) 11 / 12',
         "if x and y are positive integers and 7 + x + y + xy = 21 , what is the value of x + y ? 'try each answer choices . for a : 7 + 3 + xy = 21 ; xy = 11 ( impossible , 11 prime number . 1 + 11 does n ' t equal 3 ) for b : 7 + 5 + xy = 21 ; xy = 9 ( no combination of xy = 9 and x + y = 5 ) for c : 7 + 6 + xy = 21 ; xy = 8 ( x + y = 6 ; x = 2 , y = 4 or x = 4 , y = 2 ) for d : 7 + 8 + xy = 21 ; xy = 6 ( no combination of xy = 6 and x + y = 8 ) for e : 7 + 9 + xy = 21 ; xy = 5 ( impossible , 5 prime number . 1 + 5 does n ' t equal 9 ).' a ) 3 , b ) 5 , c ) 6 , d ) 8 , e ) 9",
         "the hcf and lcm of two numbers m and n are respectively 6 and 210 . if m + n = 72 , then 1 / m + 1 / n is equal to answer we have , m x n = 6 x 210 = 1260 ∴ 1 / m + 1 / n = ( m + n ) / mn = 72 / 1260 = 4 / 70 = 2 / 35 correct option : d a ) 1 / 35 , b ) 3 / 35 , c ) 5 / 37 , d ) 2 / 35 , e ) none",
         "in a kilometer race , a beats b by 48 meters or 12 seconds . what time does a take to complete the race ? 'time taken by b run 1000 meters = ( 1000 * 12 ) / 48 = 250 sec . time taken by a = 250 - 12 = 238 sec .' a ) 238 sec , b ) 190 sec , c ) 667 sec , d ) 167 sec , e ) 176 sec"]

In [28]:
dataset[0]

'a 3 - digit positive integer is chosen at random . what is the probability that the product of its digits is even ? p ( three odd digits ) = 5 / 9 * 5 / 10 * 5 / 10 = 5 / 36 ( first digit can not be 0 , so we have a total of 9 digits , 4 even and 5 odd ) . therefore , p ( at least one even digit ) = 1 - 5 / 36 = 31 / 36 . a ) 1 / 2 , b ) 31 / 36 , c ) 49 / 54 , d ) 7 / 8 , e ) 11 / 12'

In [ ]:
for data in dataset:
    text_to_explain = data
    probabilities, predicted_classes = predict_proba([text_to_explain])
    exp = explainer.explain_instance(text_to_explain, lambda x: predict_proba(x)[0], num_features=6, num_samples=100, top_labels=5)
    exp.show_in_notebook(text=text_to_explain,labels=[predicted_classes[0]])
    exp.show_in_notebook(text=False,labels=[0,1,2,3,4])